In [33]:
from bs4 import BeautifulSoup
import pandas as pd
import csv
import re
import time
import urllib
import requests

In [34]:
base_url = 'https://www.classic-trader.com'

In [35]:
num_Of_Page = 14

In [36]:
base_page_link = 'https://www.classic-trader.com/uk/motorcycle/search?yearBeginOfProduction=1995&priceMin=2500&priceMax=500000&currencyCode=gbp&pagination%5Bpage%5D='
products_pages = []
for i in range(num_Of_Page):
    url = f"{base_page_link}{i + 1}"
    #print(url)
    products_pages.append(url)

In [37]:
#products_pages

In [38]:
products_link_tails = []
cnt = 0
for url in products_pages:
    base_page = requests.get(url)
    soup = BeautifulSoup(base_page.text, "html")
    #find link of each product and store in an arr
    element = soup.find_all("div", {"data-link": True})
    for val in element:
        cnt+=1
        if cnt % 2 != 0:
          link = val["data-link"]
          products_link_tails.append(link)
    print("Successfully !!!")

Successfully !!!
Successfully !!!
Successfully !!!
Successfully !!!
Successfully !!!
Successfully !!!
Successfully !!!
Successfully !!!
Successfully !!!
Successfully !!!
Successfully !!!
Successfully !!!
Successfully !!!
Successfully !!!


In [39]:
#products_link_tails

In [40]:
products_link = []
for link in products_link_tails:
    products_link.append(urllib.parse.urljoin(base_url,link))

In [41]:
len(products_link)

206

In [42]:
#get columns name
url_sample = 'https://www.classic-trader.com/uk/motorcycle/listing/henderson/de-luxe-1300/1924/357180'

page = requests.get(url_sample)
soup = BeautifulSoup(page.text, 'html')
col_txt = soup.find_all('dt',limit=27) #27 columns
columns = []
for val in col_txt:
    columns.append(val.text)

columns.append('Price')


In [43]:
def convert_string(s):

    s = re.sub(r'\([^)]*\)', '', s)

    s = s.rstrip()

    s = s.replace(' ', '-')
    return s


In [44]:
columns = [convert_string(s) for s in columns]

In [45]:
csv_filename = "PKL_dataset.csv"
with open(csv_filename, mode = 'w', newline= '') as file:
    writer = csv.writer(file)
    writer.writerow(columns)
    for url in products_link:
        page = requests.get(url)
        time.sleep(3)
        soup = BeautifulSoup(page.text, 'html')
        price = soup.find('span', class_ = "text-2xl font-semibold lg:text-3xl")
        price = price.text[2:]
        val_txt = soup.find_all('dd',limit=27)
        row_data = []
        for item in val_txt:
            row_data.append(item.get_text(strip = True))
        row_data.append(price)
        writer.writerow(row_data)